In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import tensorflow as tf

tf.__version__

In [ ]:
#Part 1-Building CNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

#Initializing CNN
classifier = Sequential()

#Step 1-Convolution
classifier.add(Conv2D(32,(3,3),input_shape=(128,128,3),activation='relu'))
#Step 2-Pooling
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Conv2D(64,(3,3),activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Dropout(0.5))

classifier.add(Conv2D(128,(3,3),activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Dropout(0.5))

#Step 3-Flattening
classifier.add(Flatten())

#Step 4-Full connection
classifier.add(Dense(128, activation='relu'))
classifier.add(Dense(1, activation='sigmoid'))

In [ ]:
#Compiling CNN
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
#Part 2-Fitting CNN to images
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('/kaggle/input/dogs-cats-images/dog vs cat/dataset/training_set',
                                                 target_size=(128, 128), 
                                                 batch_size=32,#32
                                                 class_mode='binary')

test_set = test_datagen.flow_from_directory('/kaggle/input/dogs-cats-images/dog vs cat/dataset/test_set',
                                             target_size=(128, 128),
                                             batch_size=32,#32
                                             class_mode='binary')

# Early stopping

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

checkpoint_path = '/kaggle/working/best_dogs_cats_model.h5'

callbacks = [EarlyStopping(monitor='val_loss', patience=8, mode='min',restore_best_weights=True),
             ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', save_best_only=True, mode='min')]

In [ ]:
classifier.fit_generator(training_set,
                         steps_per_epoch=250,#8000
                         epochs=40,#15
                         callbacks=callbacks,
                         validation_data=test_set,
                         validation_steps=63)#2000